Accessing the Data from WRDS in Python. 

In [1]:
import wrds
import pandas as pd


Indeed, we need to have valid access to WRDS. In the first instance, we need to pass a username and password. We can save it create_pgpass_file; this way, we will not need to enter username and password.
 
I name the connection 'db' to use in the following requests. 

In [2]:
db = wrds.Connection(wrds_username='ghal')

Loading library list...
Done


In [ ]:
# Creating pgpass file. So that we don't have to enter a password every time we run the script.
db.create_pgpass_file()
dp.close()

Check the available library on wrds

In [3]:
# list of available library
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'audit_europe',
 'audit_sedar',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'block',
 'block_all',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compg',
 'compm',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_as_filed

### Extract the data:
there are two ways to extract the data. 
1) using tables.

In [ ]:
# Extract first 5 obs from crsp.crsp_daily_data
dsenames = db.get_table(library='crsp', table='dsenames')
dsenames_df = pd.DataFrame(dsenames)
dsenames_df.head(5)

,permno,namedt,nameendt,shrcd,exchcd,siccd,ncusip,ticker,comnam,shrcls,...,naics,primexch,trdstat,secstat,permco,compno,issuno,hexcd,hsiccd,cusip
0,10000,1986-01-07,1986-12-03,10,3,3990,68391610,OMFGA,OPTIMUM MANUFACTURING INC,A,...,None,Q,A,R,7952,60007905,10396,3,3990,68391610
1,10000,1986-12-04,1987-03-09,10,3,3990,68391610,OMFGA,OPTIMUM MANUFACTURING INC,A,...,None,Q,A,R,7952,60007905,10396,3,3990,68391610
2,10000,1987-03-10,1987-06-11,10,3,3990,68391610,OMFGA,OPTIMUM MANUFACTURING INC,A,...,None,Q,A,R,7952,60007905,10396,3,3990,68391610
3,10001,1986-01-09,1993-11-21,11,3,4920,39040610,GFGC,GREAT FALLS GAS CO,None,...,None,Q,A,R,7953,60007906,10398,2,4925,36720410
4,10001,1993-11-22,2004-06-09,11,3,4920,29274A10,EWST,ENERGY WEST INC,None,...,None,Q,A,R,7953,60007906,10398,2,4925,36720410


In [ ]:
# Extract first 100 obs from comp.company
comp = db.get_table(library='comp', table='company', obs = 100)
comp_df = pd.DataFrame(comp)
comp_df.head(5)

,conm,gvkey,add1,add2,add3,add4,addzip,busdesc,cik,city,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,A & E PLASTIK PAK INC,001000,None,None,None,None,None,A & E Plastik Pak Inc. is a commodity chemical...,None,None,...,01,3089,325,978,None,None,0,None,1978-06-30,None
1,A & M FOOD SERVICES INC,001001,1924 South Utica Avenue,None,None,None,94104,None,0000723576,Tulsa,...,01,5812,420,978,None,OK,0,None,1986-07-31,None
2,AAI CORP,001002,124 Industry Lane,None,None,None,21030-0126,"Textron Systems Corporation designs, develops,...",0001306124,Hunt Valley,...,01,3825,230,940,None,MD,0,www.textronsystems.com,1977-03-31,None
3,A.A. IMPORTING CO INC,001003,7700 Hall Street,None,None,None,63125,"A.A. Importing Company, Inc. designs, manufact...",0000730052,St. Louis,...,01,5712,449,976,None,MO,3,www.aaimporting.com,1992-04-30,None
4,AAR CORP,001004,"One AAR Place, 1100 North Wood Dale Road",None,None,None,60191,AAR Corp. provides products and services to co...,0000001750,Wood Dale,...,01,5080,110,925,B,IL,0,www.aarcorp.com,None,1972-04-24


2) Raw_sql() for subsetting data
We can write SQL style to extract the data. This would be helpful using larger dataset.

In [18]:
# select monthly return from for only one stock 
tesla = db.raw_sql("""select permno, date, prc, ret, shrout 
                        from crsp.msf 
                        where permno = 93436
                        and date>='01/01/2020'""", 
                     date_cols=['date'])
tesla.head(5)

,permno,date,prc,ret,shrout
0,93436,2022-08-31,275.60999,-0.072489,3133470.0
1,93436,2020-01-31,650.57001,0.555160,181062.0
2,93436,2020-02-28,667.98999,0.026777,184110.0
3,93436,2020-03-31,524.00000,-0.215557,185000.0
4,93436,2020-04-30,781.88000,0.492137,185371.0


We can make more complex queries using raw_sql function. Depends on the task, it might take a while.

Aboute the following code: I only select equity incorporated in US (shrcd equal 10 and 11) listed in NYSE, AMEX, or NASDAQ exchcd in (1, 2, 3). I also select the stock return from 2000 just to make it easier to work with the data. 

In [4]:
history_data = db.raw_sql(""" select a.permno, b.date, b.ret
                            from (select * from crsp.dsenames where shrcd in (10, 11) and exchcd in (1, 2, 3)) as a,
                            (select * from crsp.dsf) as b
                            where a.permno = b.permno and 
                            a.namedt <= b.date and b.date <= a.nameendt
                            and b.date >='01/01/2000'
                            """, date_cols = ['date'])

In [5]:
history_data.head(5)

,permno,date,ret
0,10001,2000-01-03,0.007353
1,10001,2000-01-04,-0.014599
2,10001,2000-01-05,0.014815
3,10001,2000-01-06,-0.007299
4,10001,2000-01-07,-0.007353


In [6]:
history_data.shape

(26188411, 3)

In [7]:
# save the data to a csv file
history_data.to_csv('history_data.csv', index = False)